The notebook is to work my entire workflow. 

In [1]:
from lib.func import create_accession_list_for_genbank_IDs
from pathlib import Path
create_accession_list_for_genbank_IDs(Path('/media/felix/ShortTerm/universal_makers/trna_Q8XBK3/uniprotkb_uniref_cluster_90_UniRef90_Q8_2024_09_25.fasta'), 
                                      Path('/media/felix/ShortTerm/universal_makers/trna_Q8XBK3/accession_list.txt'))

In [2]:
import os
excute_command = '/home/felix/anaconda3/envs/Get3DStructuresFixed/bin/python /media/felix/ShortTerm/Data/PycharmProjects/Get3DStructures/main.py'
for cluster_name in os.listdir('/media/felix/ShortTerm/uniref90_clusters_with_binding_site/done/'):
    
    #os.system(f"{excute_command} /media/felix/ShortTerm/uniref90_clusters_with_binding_site/job1/'{cluster_name}'/ pdb repair /media/felix/ShortTerm/uniref90_clusters_with_binding_site/job1/{cluster_name} ")
    
   # try:
   #     os.mkdir(f'/media/felix/ShortTerm/uniref90_clusters_with_binding_site/job1/{cluster_name}/aligned')
   # except:
   #     print("already here")
   # os.system(f'{excute_command} /media/felix/ShortTerm/uniref90_clusters_with_binding_site/job1/{cluster_name}/  pdb calculate align-data full homology-structure backbone /media/felix/ShortTerm/uniref90_clusters_with_binding_site/job1/{cluster_name}/aligned/')
   # os.system(f'{excute_command} /media/felix/ShortTerm/uniref90_clusters_with_binding_site/job1/{cluster_name}/aligned/  pdb calculate get-mesh full homology-structure backbone /media/felix/ShortTerm/uniref90_clusters_with_binding_site/job1/{cluster_name}/aligned/')
    os.system(f'{excute_command} /media/felix/ShortTerm/uniref90_clusters_with_binding_site/done/{cluster_name}/aligned/  pdb calculate molecular-partition full homology-structures backbone /media/felix/ShortTerm/uniref90_clusters_with_binding_site/done/{cluster_name}/aligned/')
    
    
    

[14:33:51] INFO     Copying PLDDT                             ]8;id=884819;file:///media/felix/ShortTerm/Data/PycharmProjects/Get3DStructures/dataStructure/collections.py\collections.py]8;;\:]8;id=239557;file:///media/felix/ShortTerm/Data/PycharmProjects/Get3DStructures/dataStructure/collections.py#138\138]8;;\
           INFO     Copying PLDDT                             ]8;id=219037;file:///media/felix/ShortTerm/Data/PycharmProjects/Get3DStructures/dataStructure/collections.py\collections.py]8;;\:]8;id=582901;file:///media/felix/ShortTerm/Data/PycharmProjects/Get3DStructures/dataStructure/collections.py#138\138]8;;\
           INFO     Copying PLDDT                             ]8;id=40054;file:///media/felix/ShortTerm/Data/PycharmProjects/Get3DStructures/dataStructure/collections.py\collections.py]8;;\:]8;id=833671;file:///media/felix/ShortTerm/Data/PycharmProjects/Get3DStructures/dataStructure/collections.py#138\138]8;;\
           INFO     Copying PLDDT     

In [1]:
from scipy.spatial.distance import cdist
import numpy as np
def build_set_comparison(reference, target):
    reference  = np.unique(reference,axis=0)
    target = np.unique(target,axis=0)
    #if len(reference) > len(target):
    #    reference, target = target, reference
    distance_matrix = cdist(reference[:,[4,5,6]], target[:,[4,5,6]])
    minimum_distance_index = np.argmin(distance_matrix,axis=1)
    hits, counts = np.unique(minimum_distance_index, return_counts=True)
    counts_indx = np.where(counts>1)[0]
    neighbors = np.ones(shape=(reference.shape[0]),dtype=int,)
    neighbors = neighbors*-1
    exaustive_check_count =0
    while(len(counts_indx) > 0 and exaustive_check_count < 5):
        dups= hits[counts_indx]
        for duplicate in  dups:
            dups_indx = np.where(duplicate == minimum_distance_index)[0]
            indx_to_increase_value = np.delete(dups_indx , np.argmin(distance_matrix[dups_indx, duplicate]))
            distance_matrix[indx_to_increase_value, duplicate ] +=1000
        minimum_distance_index = np.argmin(distance_matrix,axis=1)
        hits, counts = np.unique(minimum_distance_index, return_counts=True)
        counts_indx = np.where(counts>1)[0]
        exaustive_check_count +=1
    
    indx = np.array([i for i in range(reference.shape[0])])
    restriction = distance_matrix[indx, minimum_distance_index] < 2
    restriction_indx = np.where(restriction == True)
    neighbors[restriction_indx] = minimum_distance_index[restriction_indx]
    no_match_indx= np.where(neighbors == -1)[0] 
    matches = target[neighbors.T].reshape(neighbors.T.shape[0],
                                                                   1 * 10)
    matches[no_match_indx] = [-10, -10, -10, -10, -10, -10, -10, -10, -10, -10] #reference[no_match_indx]
    feature = np.concatenate((reference[indx], matches), axis=1)
    return feature

In [2]:
def structural_high_dimension_space_l2(reference, query_patch):
    reference_comparison = build_set_comparison(reference, query_patch)
    res = np.linalg.norm(reference_comparison[:,[0,1,2,3]] - reference_comparison[:,[10,11,12,13]], axis=1)
    return res

In [3]:
def jaccard_similarity(ref, target):
    
    comparison = build_set_comparison(ref, target)
    a_intersection_b = np.count_nonzero(np.linalg.norm(comparison[:,[0,1,2,3]] - comparison[:,[10,11,12,13]], axis=1)<=1)
    #print(a_intersection_b)
    a_or_b = np.unique(ref,axis=0).shape[0]+ np.unique(target,axis=0).shape[0] - a_intersection_b
    jaccrad_sim = a_intersection_b/a_or_b
    return jaccrad_sim

In [32]:
import numpy as np
import plotly.express as plt
import pandas as pd

vecs = []
ids = []
sims = []
included_set = []
excluded_set = []
ignored_set = []
from dataStructure.collections import Collection, HomologyStructureFetcher
from pathlib import Path

database_path = Path(
    '/media/felix/ShortTerm/universal_makers/non_redundant_B7LSS5/aligned/')
my_collection = Collection(database_path, HomologyStructureFetcher())
representative_patches = np.load('/media/felix/ShortTerm/universal_makers/non_redundant_B7LSS5/aligned/P59632/sp_P59632_AP3A_SARS_ORF3a_protein_OS_Severe_acute_respiratory_syndrome_coronavirus_OX_694009_GN_3a_PE_1_SV_1_relaxed_rank_001_alphafold2_ptm_model_3_seed_000-full-patches-with-vertex-pos.npy')
funsocDB = open('/media/felix/ShortTerm/experemental_verified_funsocs/accession_list.txt','r')
funsocEntires = funsocDB.read().splitlines()
funsoc = ""
for structures in my_collection.protein_structure_results.values():
    for structure in structures.all_structures:
        ids.append(structure.id)
        target = np.load(structure.path.parent.joinpath(structure.path.name.split(".")[0] + "-full-patches-with-vertex-pos.npy"))
        similarity = jaccard_similarity(representative_patches, target)
        sims.append(similarity)
        if structure.id in ['P03407']:
            swiss_prot = 'Swiss-prot'
        else:
            swiss_prot = 'Non Swiss-prot'
        if structure.id in funsocEntires:
            funsoc = "FunSoC"
        else:
            funsoc="Non-FunSoC"

        included_set.append((swiss_prot ,structure.id, similarity, funsoc))
df = pd.DataFrame(included_set, columns=['Dataset', 'Name','Similarity', "FunSoC"])
fig = plt.scatter(df, x='Name' ,y='Similarity',title="Jaccard Similarity of Vertex Aligned Structures Of cluster_P59632",color='FunSoC',)
fig.show()


In [35]:
from Bio import pairwise2, Align
import numpy as np
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import plotly.express as plt
matrix = Align.substitution_matrices.load("BLOSUM62")
vecs = []
ids = []
swiss = []
enties = []
seq_ids = []
for structures in my_collection.protein_structure_results.values():
    for structure in structures.all_structures:
        ids.append(structure.id)
        target = np.load(structure.path.parent.joinpath(structure.path.name.split(".")[0] + "-full-patches-with-vertex-pos.npy"))
        vecs.append(structural_high_dimension_space_l2(representative_patches, target))
        if structure.id in ['P77335', 'Q9REB3', 'Q8Z727','Q93RR6']:
            swiss.append('Swiss-prot')
        else:
            swiss.append('Non Swiss-prot')
tsne = TSNE()
pca = PCA()
print(seq_ids)
ids = np.array(ids)
embedding = pca.fit_transform(np.array(vecs))
df['embedding0'] = embedding[:,0] 
df['embedding1']=embedding[:,1]
#print(df)
#print(df['embedding0'])x='Name' ,y='Similarity'
fig=plt.scatter(df, x='embedding0'  , y='embedding1', labels=['Similarity'], title="PCA of L2 Distance of Vertex Features from cluster_P59632.", color='Similarity',color_continuous_scale='rainbow')

fig.show()
fig.write_image('/home/felix/cluster_P59632_pca.png')

[]
